<a href="https://colab.research.google.com/github/rishisaxena300/Agentic-AI-Learning/blob/main/LLM_Tool_Calling_Function_Calling_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import json
from openai import OpenAI
from google.colab import userdata
import os

# --------------------------------------------------
# Initialize OpenAI client
# Make sure your OPENAI_KEY is set in Colab secrets:
# 1. Click the 'key' icon on the left panel.
# 2. Add a new secret with the name 'OPENAI_KEY' and your API key as its value.
# --------------------------------------------------

# Retrieve API key from Colab secrets
openai_api_key = userdata.get('OPENAI_KEY')

# Initialize OpenAI client with the retrieved key
client = OpenAI(api_key=openai_api_key)

# --------------------------------------------------
# Problem:
# Count occurrences of a specific letter in a word
# using Tool (Function) Calling with an LLM
# --------------------------------------------------

# 1. Define the tool function
def letter_counter(word: str, letter: str) -> int:
    """
    Counts the occurrences of a specific letter within a word.
    Case-insensitive.
    """
    return word.lower().count(letter.lower())

# --------------------------------------------------
# 2. Define the tool schema for the LLM
# --------------------------------------------------

tools = [
    {
        "type": "function",
        "name": "letter_counter",
        "description": "Counts the occurrences of a specific letter within a word.",
        "parameters": {
            "type": "object",
            "properties": {
                "word": {
                    "type": "string",
                    "description": "The word in which to count the letter"
                },
                "letter": {
                    "type": "string",
                    "description": "The letter to count"
                }
            },
            "required": ["word", "letter"]
        }
    }
]

# --------------------------------------------------
# 3. Initial user prompt
# --------------------------------------------------

user_prompt = (
    "Use the letter_counter tool to find out how many "
    "'r' are there in the word 'Strawberry'."
)

input_list = [
    {
        "role": "user",
        "content": user_prompt
    }
]

print("Initial input list:\n", input_list)

# --------------------------------------------------
# 4. Ask the model to decide whether a tool is needed
# --------------------------------------------------

response = client.responses.create(
    model="gpt-4o-mini",
    tools=tools,
    input=input_list,
)

print("\nModel tool call response:\n", response.output)

# Save tool call into conversation history
input_list += response.output

# --------------------------------------------------
# 5. Extract arguments from the tool call
# --------------------------------------------------

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

print("\nTool arguments extracted:\n", args)

# --------------------------------------------------
# 6. Execute the tool function
# --------------------------------------------------

letter_count = letter_counter(
    word=args["word"],
    letter=args["letter"]
)

print(
    f"\nletter_counter result for '{args['letter']}' "
    f"in '{args['word']}': {letter_count}"
)

# --------------------------------------------------
# 7. Send tool output back to the LLM
# --------------------------------------------------

input_list.append(
    {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": json.dumps({"letter_count": letter_count})
    }
)

print("\nInput list after tool execution:\n", input_list)

# --------------------------------------------------
# 8. Get final user-friendly response
# --------------------------------------------------

final_response = client.responses.create(
    model="gpt-4o-mini",
    instructions="Respond only with the result generated by the tool in a user-friendly sentence.",
    tools=tools,
    input=input_list,
)

print("\nFinal response from the model:\n")
print(final_response.output_text)


Initial input list:
 [{'role': 'user', 'content': "Use the letter_counter tool to find out how many 'r' are there in the word 'Strawberry'."}]

Model tool call response:
 [ResponseFunctionToolCall(arguments='{"word":"Strawberry","letter":"r"}', call_id='call_Q4cAD9Q8HYj32MexieKC62yl', name='letter_counter', type='function_call', id='fc_046de235816945fd006946bd425b98819d96fe283b225dd353', status='completed')]

Tool arguments extracted:
 {'word': 'Strawberry', 'letter': 'r'}

letter_counter result for 'r' in 'Strawberry': 3

Input list after tool execution:
 [{'role': 'user', 'content': "Use the letter_counter tool to find out how many 'r' are there in the word 'Strawberry'."}, ResponseFunctionToolCall(arguments='{"word":"Strawberry","letter":"r"}', call_id='call_Q4cAD9Q8HYj32MexieKC62yl', name='letter_counter', type='function_call', id='fc_046de235816945fd006946bd425b98819d96fe283b225dd353', status='completed'), {'type': 'function_call_output', 'call_id': 'call_Q4cAD9Q8HYj32MexieKC62yl'